In [ ]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Reload environment
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
import torch
from torchtext.legacy import data
import torch.nn as nn
import re
import nltk
import pandas as pd
from sklearn.datasets import load_files
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import urllib.request as ur
nltk.download('wordnet')
nltk.download('omw-1.4')
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt



import pickle


#Label = pickle.loads(f)

#with open("FinalLink", "rb") as fp:   # Unpickling

#  ValidLink = pickle.load(fp)

with open("Fianlcontents", "rb") as fp:   # Unpickling

  contents = pickle.load(fp)

with open("FinalLabel", "rb") as fp:   # Unpickling

  Label = pickle.load(fp)



import pandas as pd  
  
     
# list of name, degree, score 
text = ["text", "labels"] 
label = [0,1] 

     
# dictionary of lists  
dict = {'text':contents, 'labels':Label}  
       
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv('Finaltext.csv')

In [ ]:
#test = pd.read_csv("web.csv")
train = pd.read_csv("Finaltext.csv")

train.head
#print(train.target)


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(contents,Label,test_size=0.35,shuffle=True)

dict = {'text':list(x_train), 'labels':list(y_train)}  
  
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df=df.to_csv('train.csv')



dict = {'text':list(x_test), 'labels':list(y_test)}  
  
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df=df.to_csv('test.csv')



train=pd.read_csv('train.csv')

test=pd.read_csv('train.csv')

# New Section

In [ ]:
# drop 'id' , 'keyword' and 'location' columns.
#train.drop(columns=['id','keyword','location'], inplace=True)

In [ ]:
# to clean data
def removeDigits(List):
    
    answer = []
    for s in List:
      res=""
      for char in s :
        if not char.isdigit() and char!=' ':
              res+=char
      if len(res)>=2:
        answer.append(res)
    return ' '.join(answer)


def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [ ]:
train["text"]=normalise_text(train["text"])

test["text"]=normalise_text(test["text"])

In [ ]:
# split data into train and validation 
train_df, valid_df = train_test_split(train)



In [ ]:
SEED = 2022

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

print(LABEL)

In [ ]:
class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.labels if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), data_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [ ]:
fields = [('text',TEXT), ('label',LABEL)]

train_ds, val_ds = DataFrameDataset.splits(fields, train_df=train_df, val_df=valid_df)



In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_ds, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = 'glove.6B.200d',
                 unk_init = torch.Tensor.zero_)

In [ ]:
 
LABEL.build_vocab(train_ds)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi) 


# load pickle module
import pickle



# create a binary pickle file 
f = open("file.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(TEXT.vocab.stoi,f)

# close file
f.close()


In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_ds, val_ds), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)


In [ ]:
# Hyperparameters
num_epochs = 15
learning_rate = 0.001

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding

In [ ]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]

        print(text.shape)

        print(text_lengths.shape)
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output))
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [ ]:
model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
#  to initiaise padded to zeros
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data.shape)

In [ ]:
model.to(device) #CNN to GPU


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    print(rounded_preds)
    correct = (rounded_preds == y).float() #convert into float for division 

    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# training function 
def train(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text    
        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)



In [ ]:

def evaluate(model, iterator):
    count=0    
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            
            text, text_lengths = batch.text
            print(text.shape,text_lengths.shape)
              
            predictions = model(text, text_lengths).squeeze(1)

            
            acc = binary_accuracy(predictions, batch.label)

            
            epoch_acc += acc.item()
        
    return epoch_acc / len(iterator)

In [ ]:
t = time.time()
loss=[]
acc=[]
val_acc=[]
best_train_loss = float('inf')
for epoch in range(15):
    
    train_loss, train_acc = train(model, train_iterator)
    
    valid_acc = evaluate(model, valid_iterator)
    if train_loss < best_train_loss:
        best_train_loss = train_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Acc: {valid_acc*100:.2f}%')
    
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    
print(f'time:{time.time()-t:.3f}')

In [ ]:
path="saved_weights.pt"
model.load_state_dict(torch.load(path));
model.eval();

#inference 
import spacy
nlp = spacy.load("en_core_web_sm")

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1)                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)
    prediction=torch.sigmoid(prediction)        
    print(prediction)

In [ ]:
import re
import nltk
import pandas as pd
from sklearn.datasets import load_files
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import urllib.request as ur
nltk.download('wordnet')
nltk.download('omw-1.4')
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
def removeDigits(List):
    
    answer = []
    for s in List:
      res=""
      for char in s :
        if not char.isdigit() and char!=' ':
              res+=char
      if len(res)>=2:
        answer.append(res)
    return ' '.join(answer)
#Fetch the contents from website and rmove all the unessary character,punctuations
def fetch_content(url):

      if "http://" and "https://" not in url:
        url="http://"+url
      X=[]

      try:
        page = requests.get(url)

        soup = BeautifulSoup(page.text, 'html.parser')

        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        # get text
        text = soup.get_text()
        text=re.sub('[^A-Za-z0-9]+', ' ', text).strip()
        text=str(text).replace('\n',' ').split(' ')

        for i in text:
          if i:
            X.append(i)
      except:
        X=[]

      documents = []
      stemmer = WordNetLemmatizer()
      print(X)
      if X:
        #Remove the digits from string:
        document = removeDigits(X)

        # Remove all the special characters
        document = re.sub(r'\W', ' ', document)

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'^[a-zA-Z]\s+', ' ', document) 

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)

        return document

In [ ]:
content=fetch_content("https://phrma.org/states?utm_campaign=2022-q4-pbm-afd&utm_medium=pai_srh_cpc-ggl-adf&utm_source=ggl&utm_content=clk-pat-tpv_scl-geo_std-usa-nj-pai_srh_cpc-ggl-adf-NewJerseyStatePBMsSearchWC2-prc_brr-edu-inf-lrm-soc_txt-std-vrb-art&utm_term=&https://p2a.co/pqccffz&gclid=EAIaIQobChMIhJy5kPP1-wIVGMzICh02VwyAEAAYASAAEgJvaPD_BwE")

print(content)


predict(model, content)

In [ ]:
for batch in train_iterator:
    
    print(batch.text[0].shape)
    break